# Проект по SQL: Анализ базы данных сервиса для чтения книг по подписке

**1. Описание проекта**

Проект заключается в анализе базы данных крупного сервиса для чтения книг по подписке. База данных содержит информацию о книгах, издательствах, авторах, а также пользовательские обзоры и оценки книг. Данные будут использованы для формулирования ценностного предложения нового продукта.

**2. Цели и задачи проекта**

**Цели проекта:**
- Анализ текущего состояния рынка книг по подписке.
- Определение ключевых факторов успеха для нового продукта.

**Задачи проекта:**
- Посчитать количество книг, вышедших после 1 января 2000 года.
- Для каждой книги посчитать количество обзоров и среднюю оценку.
- Определить издательство, выпустившее наибольшее число книг толще 50 страниц.
- Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками.
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

**Описание данных**

*Таблица books*

Содержит данные о книгах:
  - book_id — идентификатор книги;
  - author_id — идентификатор автора;
  - title — название книги;
  - num_pages — количество страниц;
  - publication_date — дата публикации книги;
  - publisher_id — идентификатор издателя.

*Таблица authors*

Содержит данные об авторах:
  - author_id — идентификатор автора;
  - author — имя автора.

*Таблица publishers*

Содержит данные об издательствах:
  - publisher_id — идентификатор издательства;
  - publisher — название издательства;

*Таблица ratings*

Содержит данные о пользовательских оценках книг:
  - rating_id — идентификатор оценки;
  - book_id — идентификатор книги;
  - username — имя пользователя, оставившего оценку;
  - rating — оценка книги.
  
*Таблица reviews*

Содержит данные о пользовательских обзорах:
  - review_id — идентификатор обзора;
  - book_id — идентификатор книги;
  - username — имя автора обзора;
  - text — текст обзора.

**3. Ход работы**

**Этап 1: Подготовка и исследование данных**
- Подключиться к базе данных.
- Исследовать таблицы, вывести первые строки для ознакомления с данными.

**Этап 2: Выполнение SQL-запросов**
- **Задача 1:** Посчитать количество книг, вышедших после 1 января 2000 года.

- **Задача 2:** Для каждой книги посчитать количество обзоров и среднюю оценку.

- **Задача 3:** Определить издательство, выпустившее наибольшее число книг толще 50 страниц.

- **Задача 4:** Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками.
 
- **Задача 5:** Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.
 
Вывести результаты каждого запроса в тетрадке.
Описать выводы по каждой из решённых задач.

**Этап 3: Вывод**

## Подготовка и исследование данных

**Подключение к базе данных**

Для начала подключим необходимые библиотеки и загрузим данные

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

**Исследование таблиц**

Для начала посмотрим размеры таблиц

In [3]:
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']

with engine.connect() as con:
    for table in tables:
        size_query = f'SELECT COUNT(*) FROM {table}'
        size_df = pd.io.sql.read_sql(sql=text(size_query), con=con)
        table_size = size_df.iloc[0, 0]
        print(f"Размер таблицы '{table}': {table_size}")

Размер таблицы 'books': 1000
Размер таблицы 'authors': 636
Размер таблицы 'publishers': 340
Размер таблицы 'ratings': 6456
Размер таблицы 'reviews': 2793


Основываясь на предоставленных данных о размерах таблиц, можно сделать следующие выводы:

1. **Таблица 'books'**: Содержит 1000 записей. Это количество книг в базе данных.

2. **Таблица 'authors'**: Содержит 636 записей. Это количество авторов в базе данных. Учитывая, что количество авторов меньше количества книг, можно предположить, что многие авторы написали более одной книги.

3. **Таблица 'publishers'**: Содержит 340 записей. Это количество издательств в базе данных. Это число значительно меньше количества книг, что указывает на то, что многие издательства выпустили несколько книг.

4. **Таблица 'ratings'**: Содержит 6456 записей. Это количество оценок, которые пользователи поставили книгам. Это число значительно превышает количество книг, что говорит о том, что многие книги получили несколько оценок.

5. **Таблица 'reviews'**: Содержит 2793 записей. Это количество обзоров, оставленных пользователями. Это число меньше количества оценок, что может означать, что не все пользователи оставляют обзоры, или что не все книги получили обзоры.



Далее выведем первые строки каждой таблицы.

**Таблица books**

In [4]:
query = '''SELECT * 
           FROM books
           LIMIT 5;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Таблица authors**

In [5]:
query = '''SELECT * 
           FROM authors 
           LIMIT 5;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица publishers**

In [6]:
query = '''SELECT * 
           FROM publishers 
           LIMIT 5;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Таблица ratings**

In [7]:
query = '''SELECT * 
           FROM ratings 
           LIMIT 5;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Таблица reviews**

In [8]:
query = '''SELECT * 
           FROM reviews 
           LIMIT 5;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Вывод:**

На данном этапе работы были подключены библиотеки и загруженны данные. При рассмотрении данных можно заметить:

- Многие авторы и издательства имеют несколько книг в базе данных.
- Книги часто получают несколько оценок, что указывает на активность пользователей в оценке книг.
- Количество обзоров меньше количества оценок, что может указывать на более низкую активность пользователей в написании обзоров по сравнению с оценкой книг.

Эти выводы могут быть полезны для понимания структуры данных и активности пользователей в сервисе для чтения книг по подписке.

## Выполнение SQL-запросов

# Задача 1: Посчитать количество книг, вышедших после 1 января 2000 года.



In [9]:
query = """SELECT COUNT(*) AS books_after_2000 
         FROM books 
         WHERE publication_date > '2000-01-01';"""

data = pd.read_sql(query, engine)

print(data)

   books_after_2000
0               819


После 1 января 2000 года вышло 819 книг.

# Задача 2: Для каждой книги посчитать количество обзоров и среднюю оценку.



In [21]:
query = """
SELECT b.book_id, b.title, 
COUNT(DISTINCT r.review_id) AS review_count, 
AVG(rt.rating) AS avg_rating
FROM books b
LEFT JOIN reviews r ON b.book_id = r.book_id
LEFT JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY b.book_id, b.title;
"""

data = pd.read_sql(query, engine)

data = data.sort_values(by='avg_rating', ascending=False)
display(data)

,book_id,title,review_count,avg_rating
901,902,The War of Art: Break Through the Blocks & Win...,2,5.00
937,938,Triptych (Will Trent #1),2,5.00
517,518,Pop Goes the Weasel (Alex Cross #5),2,5.00
168,169,Crucial Conversations: Tools for Talking When ...,2,5.00
731,732,The Ghost Map: The Story of London's Most Terr...,2,5.00
...,...,...,...,...
914,915,The World Is Flat: A Brief History of the Twen...,3,2.25
315,316,His Excellency: George Washington,2,2.00
370,371,Junky,2,2.00
201,202,Drowning Ruth,3,2.00


In [22]:
max_avg_rating = data['avg_rating'].max()
min_avg_rating = data['avg_rating'].min()
max_review_count = data['review_count'].max()
min_review_count = data['review_count'].min()

print(f"Максимальный средний рейтинг: {max_avg_rating}")
print(f"Минимальный средний рейтинг: {min_avg_rating}")
print(f"Максимальное количество обзоров: {max_review_count}")
print(f"Минимальное количество обзоров: {min_review_count}")

Максимальный средний рейтинг: 5.0
Минимальный средний рейтинг: 1.5
Максимальное количество обзоров: 7
Минимальное количество обзоров: 0


In [23]:
total_reviews_query = """SELECT COUNT(DISTINCT review_id) AS total_reviews 
                         FROM reviews;"""
total_reviews_data = pd.read_sql(total_reviews_query, engine)
total_reviews_from_db = total_reviews_data['total_reviews'].iloc[0]

total_reviews_from_data = data['review_count'].sum()

print(f"Общее количество отзывов в базе данных: {total_reviews_from_db}")
print(f"Сумма по столбцу review_count в DataFrame: {total_reviews_from_data}")

if total_reviews_from_db == total_reviews_from_data:
    print("Сумма по столбцу review_count совпадает с общим количеством отзывов в базе данных.")
else:
    print("Сумма по столбцу review_count не совпадает с общим количеством отзывов в базе данных.")

Общее количество отзывов в базе данных: 2793
Сумма по столбцу review_count в DataFrame: 2793
Сумма по столбцу review_count совпадает с общим количеством отзывов в базе данных.


Сумма по столбцу review_count не совпадает с общим количеством отзывов в базе данных, это может указывать на несовершенство данных и возможные ошибки при формировании таблицы

Вывод по итогу анализа:

1. **Количество обзоров и средний рейтинг**:
   - Было проанализировано 1000 книг.
   - Максимальный средний рейтинг составил 5.0, а минимальный — 1.5.
   - Максимальное количество обзоров для одной книги составило 7, а минимальное — 0.

2. **Совпадение суммы обзоров**:
   - Общее количество уникальных отзывов в базе данных составило 2793.
   - Сумма количества обзоров по всем книгам в DataFrame также составила 2793.
   - Это свидетельствует о том, что данные в DataFrame корректно отражают общее количество отзывов в базе данных.

3. **Высокий средний рейтинг**:
   - Некоторые книги имеют высокий средний рейтинг (5.0), что может указывать на их популярность и положительные отзывы читателей.

4. **Низкий средний рейтинг**:
   - Некоторые книги имеют низкий средний рейтинг (1.5), что может свидетельствовать о негативном восприятии этих книг читателями.

5. **Разброс количества обзоров**:
   - Существует значительный разброс в количестве обзоров для разных книг, от 0 до 7. Это может указывать на различную популярность книг среди читателей.

В целом, анализ показал, что данные о количестве обзоров и средних оценках книг корректно отражаются в DataFrame и соответствуют данным в базе данных. Это позволяет сделать вывод о надежности и точности представленных данных.

# Задача 3: Определить издательство, выпустившее наибольшее число книг толще 50 страниц.


In [16]:
query = """
SELECT 
    p.publisher_id,
    p.publisher,
    COUNT(b.book_id) AS book_count
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id, p.publisher
ORDER BY book_count DESC
LIMIT 1;
"""

data = pd.read_sql(query, engine)

display(data)

,publisher_id,publisher,book_count
0,212,Penguin Books,42


Издательство, выпустившее наибольшее число книг толще 50 страниц - Penguin Books

# Задача 4: Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками.


In [18]:
query = """
WITH book_ratings AS (
    SELECT 
        b.author_id,
        b.book_id,
        AVG(r.rating) AS avg_rating,
        COUNT(r.rating_id) AS rating_count
    FROM books b
    JOIN ratings r ON b.book_id = r.book_id
    GROUP BY b.author_id, b.book_id
    HAVING COUNT(r.rating_id) >= 50
),
author_avg_ratings AS (
    SELECT 
        br.author_id,
        AVG(br.avg_rating) AS author_avg_rating
    FROM book_ratings br
    GROUP BY br.author_id
)
SELECT 
    a.author_id,
    a.author,
    aar.author_avg_rating
FROM authors a
JOIN author_avg_ratings aar ON a.author_id = aar.author_id
ORDER BY aar.author_avg_rating DESC
LIMIT 1;
"""

data = pd.read_sql(query, engine)

display(data)

,author_id,author,author_avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.283844


Автор с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками - J.K. Rowling/Mary GrandPré.

# Задача 5: Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [19]:
query = """
WITH user_reviews AS (
    SELECT 
        r.username,
        COUNT(r.review_id) AS review_count
    FROM reviews r
    JOIN (
        SELECT 
            username
        FROM ratings
        GROUP BY username
        HAVING COUNT(rating_id) > 48
    ) u ON r.username = u.username
    GROUP BY r.username
)
SELECT AVG(review_count) AS average_reviews
FROM user_reviews;

"""

data = pd.read_sql(query, engine)

display(data)

,average_reviews
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24

## Вывод:

Вывод:

Проведенный анализ базы данных сервиса для чтения книг по подписке позволил сделать несколько важных выводов, которые могут быть полезны для понимания структуры данных, активности пользователей и качества книг в сервисе.

1. **Размеры таблиц**:
   - Таблица 'books' содержит 1000 записей, что указывает на количество книг в базе данных.
   - Таблица 'authors' содержит 636 записей, что говорит о количестве авторов. Учитывая, что количество авторов меньше количества книг, можно предположить, что многие авторы написали более одной книги.
   - Таблица 'publishers' содержит 340 записей, что указывает на количество издательств. Это число значительно меньше количества книг, что свидетельствует о том, что многие издательства выпустили несколько книг.
   - Таблица 'ratings' содержит 6456 записей, что указывает на количество оценок, которые пользователи поставили книгам. Это число значительно превышает количество книг, что говорит о том, что многие книги получили несколько оценок.
   - Таблица 'reviews' содержит 2793 записей, что указывает на количество обзоров, оставленных пользователями. Это число меньше количества оценок, что может означать, что не все пользователи оставляют обзоры, или что не все книги получили обзоры.

2. **Анализ книг, вышедших после 1 января 2000 года**:
   - После 1 января 2000 года вышло 819 книг. Это указывает на активность издательств и авторов в выпуске новых книг в последние годы.

3. **Количество обзоров и средние оценки книг**:
  
   - Было проанализировано 1000 книг.
   - Максимальный средний рейтинг составил 5.0, а минимальный — 1.5.
   - Максимальное количество обзоров для одной книги составило 7, а минимальное — 0.
   - Общее количество уникальных отзывов в базе данных составило 2793.
   - Сумма количества обзоров по всем книгам в DataFrame также составила 2793.
   - Это свидетельствует о том, что данные в DataFrame корректно отражают общее количество отзывов в базе данных.
   - Некоторые книги имеют высокий средний рейтинг (5.0), что может указывать на их популярность и положительные отзывы читателей.
   - Некоторые книги имеют низкий средний рейтинг (1.5), что может свидетельствовать о негативном восприятии этих книг читателями.
   - Существует значительный разброс в количестве обзоров для разных книг, от 0 до 7. Это может указывать на различную популярность книг среди читателей.

В целом, анализ показал, что данные о количестве обзоров и средних оценках книг корректно отражаются в DataFrame и соответствуют данным в базе данных. Это позволяет сделать вывод о надежности и точности представленных данных.

4. **Издательство с наибольшим количеством книг толще 50 страниц**:
   - Издательство "Penguin Books" выпустило наибольшее число книг толще 50 страниц. Это может свидетельствовать о популярности и активности данного издательства на рынке.

5. **Автор с самой высокой средней оценкой книг**:
   - Автор с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками, - J.K. Rowling/Mary GrandPré. Это может быть рекомендацией для читателей, ищущих книги высокого качества.

6. **Среднее количество обзоров от пользователей, которые поставили больше 48 оценок**:
   - Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет 24. Это указывает на активность пользователей в написании обзоров после того, как они оставили достаточное количество оценок.

В целом, анализ данных позволяет получить ценную информацию о структуре базы данных, активности пользователей и качестве книг в сервисе для чтения книг по подписке. Эти выводы могут быть использованы для улучшения сервиса, рекомендаций для пользователей и стратегий издательств и авторов.